In [1]:
from packages import *
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.data.experimental import AUTOTUNE
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score, r2_score
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
import re
from collections import Counter

import numpy as np
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import GlobalAveragePooling1D
from tensorflow.keras.datasets import imdb

%load_ext autoreload
%autoreload 2

In [2]:
tf.__version__

'2.0.0-alpha0'

In [3]:
epsilon = 1e-7
fp = 'float32'
tf.config.gpu.set_per_process_memory_growth(True)
tf.config.gpu.set_per_process_memory_fraction(.2)
tf.keras.backend.set_floatx(fp)
tf.keras.backend.set_epsilon(epsilon)
tf.keras.backend.set_image_data_format('channels_last')

In [32]:
batch_size = 100
glove_dim = 50
input_shape = (300, glove_dim, 1)
max_length = 300
shuffle_buffer_size = batch_size*4
prefetch_buffer_size = 1
random_seed = np.random.randint(0, 100)
test_ratio = 0.33

In [33]:
def shuffle_sample(dictionary, n_sample=None, random_seed=42):
    lens = [len(l) for l in dictionary.values()]
    assert min(lens) == max(lens)
    n_data = lens[0]
    processed = {}
    for key, array in dictionary.items():
        if n_sample is not None:
            processed[key] = shuffle(array, random_state=random_seed)[:n_sample]
        else:
            processed[key] = shuffle(array, random_state=random_seed)[:n_sample]
    return processed

In [34]:
data = shuffle_sample(load_data({'review': ['text', 'stars']})['review'],
                             random_seed=random_seed
                            )
x_train, x_test, y_train, y_test = train_test_split(data['text'],
                                                    data['stars'],
                                                    test_size = test_ratio,
                                                   )

In [35]:
glove_lookup = load_pickle(os.path.join(GLOVE_DIR, 'glove-{}D.pkl'.format(glove_dim)))
unk_key = '<UNK>'

In [36]:
unk_vector = np.mean(np.array(list(glove_lookup.values())), axis=0)
glove_lookup[unk_key] = unk_vector

In [37]:
print('Loading data...')
# (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train_tf = tf.data.Dataset.from_tensor_slices(x_train)
y_train_tf = tf.data.Dataset.from_tensor_slices(y_train - 1) # to make stars 0-indexed

x_test_tf = tf.data.Dataset.from_tensor_slices(x_test)
y_test_tf = tf.data.Dataset.from_tensor_slices(y_test - 1)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
print('Average train sequence length: {}'.format(
    np.mean(list(map(len, x_train)), dtype=int)))
print('Average test sequence length: {}'.format(
    np.mean(list(map(len, x_test)), dtype=int)))

Loading data...
4479553 train sequences
2206347 test sequences
Average train sequence length: 602
Average test sequence length: 602


In [38]:
def clean_token(t):
    t = t.lower()
    m = re.match('^[^\w\'](\w+).*', t)
    if m is not None:
        return m.group(1)
    else:
        return t

def tokenize(s):
    tokens = []
    for t in word_tokenize(s):
        tokens.append(clean_token(t))
    return tokens

# Returns an np.array of glove embeddings for each
# word in the given string of shape (word_count, glove_dims)
def get_glove_embeddings(tokens):
    embeddings = []
    
    for i, word in enumerate(tokens):
#         if i > 
        if word in glove_lookup:
            embeddings.append(glove_lookup[word])
        else:
            embeddings.append(np.zeros(glove_dim))
            
#     return(np.array(embeddings, dtype=np.float16))
    return(np.array(embeddings, dtype=np.float32))

def test_embed(arr):
    return get_glove_embeddings(tokenize(arr))

def embed(tensor):
    return get_glove_embeddings(tokenize(str(tensor.numpy())))

@tf.function
def fix_dimensions(tensor):
    return tf.image.resize_image_with_crop_or_pad(tf.expand_dims(tensor, axis=-1), 300, glove_dim)

In [39]:
train_dataset_embed = tf.data.Dataset.map(x_train_tf,  lambda review: tf.py_function( embed, [review], tf.float32 ), num_parallel_calls=AUTOTUNE) 
test_dataset_embed = tf.data.Dataset.map(x_test_tf,  lambda review: tf.py_function( embed, [review], tf.float32 ),num_parallel_calls=AUTOTUNE) 

In [40]:
train_dataset_embed = tf.data.Dataset.map(train_dataset_embed, fix_dimensions, num_parallel_calls=AUTOTUNE) 
test_dataset_embed = tf.data.Dataset.map(test_dataset_embed, fix_dimensions, num_parallel_calls=AUTOTUNE) 

In [41]:
train_dataset = tf.data.Dataset.zip((train_dataset_embed, y_train_tf))
test_dataset = tf.data.Dataset.zip((test_dataset_embed, y_test_tf))

In [42]:
train_dataset = tf.data.Dataset.shuffle(train_dataset, buffer_size=shuffle_buffer_size)
test_dataset = tf.data.Dataset.shuffle(test_dataset, buffer_size=shuffle_buffer_size)

In [43]:
train_dataset = tf.data.Dataset.batch(train_dataset, batch_size=batch_size)
test_dataset = tf.data.Dataset.batch(test_dataset, batch_size=batch_size)

In [44]:
train_dataset = tf.data.Dataset.prefetch(train_dataset, buffer_size=prefetch_buffer_size)
test_dataset = tf.data.Dataset.prefetch(test_dataset, buffer_size=prefetch_buffer_size)

# Prep Data

In [49]:
# Step 1: Get data from tensor
X = []
Y = []
for x_batch, y_batch in train_dataset.take(10000 // batch_size):
    X.append(x_batch)
    Y.append(y_batch)

In [50]:
def prep_train_tyler(X):
    data = np.dstack([X[0].numpy()[i] for i in range(X[0].shape[0])])
    for j in range(10000 // batch_size):
        if j == 0:
            continue
        data = np.append(data, np.dstack([X[j].numpy()[i] for i in range(X[j].shape[0])]), axis = 2)
    print("Shape:", data.shape)
    return(data)

def prep_test_tyler(Y):
    data = np.concatenate([Y[i].numpy() for i in range(len(Y))])
    print("Shape:", data.shape)
    return(data)

In [51]:
x_train_mdl = prep_train_tyler(X)

Shape: (300, 50, 10000)


In [52]:
x_train_mdl = x_train_mdl.T

In [53]:
y_train_mdl = prep_train_tyler(Y)

Shape: (1, 1, 10000)


In [54]:
y_train_mdl = y_train_mdl.T

In [55]:
y_train_mdl = y_train_mdl.reshape(y_train_mdl.shape[:2])

In [56]:
y_train_mdl.shape

(10000, 1)

In [57]:
x_train_mdl.shape

(10000, 50, 300)

In [58]:
x_test_mdl = x_train_mdl
y_test_mdl = y_train_mdl

In [59]:
x_test_mdl.shape

(10000, 50, 300)

In [60]:
y_test_mdl.shape

(10000, 1)

In [61]:
print('Build model...')
# Build model
model = Sequential()
model.add(Flatten())
model.add(Dense(50, activation='relu', input_shape=(15000,)))
model.add(Dense(5, activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_train_mdl, y_train_mdl,
          batch_size=batch_size,
          epochs=100,
          validation_split=.2,
#           validation_data=(x_test_mdl, y_test_mdl)
         )

Build model...
Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 2s 268us/sample - loss: 1.5222 - accuracy: 0.4235 - val_loss: 1.3532 - val_accuracy: 0.4380
Epoch 2/100
8000/8000 [==============================] - 2s 236us/sample - loss: 1.1440 - accuracy: 0.5414 - val_loss: 1.2974 - val_accuracy: 0.4705
Epoch 3/100
8000/8000 [==============================] - 2s 232us/sample - loss: 0.9163 - accuracy: 0.6417 - val_loss: 1.3209 - val_accuracy: 0.4730
Epoch 4/100
8000/8000 [==============================] - 2s 233us/sample - loss: 0.7248 - accuracy: 0.7232 - val_loss: 1.3134 - val_accuracy: 0.4650
Epoch 5/100
8000/8000 [==============================] - 2s 230us/sample - loss: 0.5652 - accuracy: 0.8001 - val_loss: 1.5967 - val_accuracy: 0.4880
Epoch 6/100
8000/8000 [==============================] - 2s 230us/sample - loss: 0.4072 - accuracy: 0.8691 - val_loss: 1.6544 - val_accuracy: 0.4835
Epoch 7/100
8000/8000 [====================

In [69]:
nsamples, nx, ny = x_train_mdl.shape
x_softmax = x_train_mdl.reshape((nsamples,nx*ny))
x_softmax.shape

(10000, 15000)

In [70]:
nsamples, nx = y_train_mdl.shape
y_softmax = y_train_mdl.reshape((nsamples,))
y_softmax.shape

(10000,)

In [78]:
x_train = x_softmax[:8000]
y_train = y_softmax[:8000]

x_test = x_softmax[8000:]
y_test = y_softmax[8000:]

In [79]:
from sklearn.linear_model import LogisticRegression

In [80]:
classifier = LogisticRegression()
classifier.fit(x_train, y_train)
score = classifier.score(x_test, y_test)

/home/kaan/anaconda3/envs/yelp-nlu_3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/kaan/anaconda3/envs/yelp-nlu_3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [81]:
print("Accuracy: ", score)

Accuracy:  0.412
